<a href="https://colab.research.google.com/github/Junjaee/Study/blob/main/R/04_DataProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### 기술 통계량 ####

### table()
aws <- read.delim("../data/AWS_sample.txt", sep = "#")
head(aws)
str(aws)

table(aws$AWS_ID)
?table
table(aws$AWS_ID, aws$X.)
View(aws)

table(aws[, c("AWS_ID", "X.")])

aws[2500:3100, "X."] = "modified"
aws[2500:3100, "X."]
table(aws$AWS_ID, aws$X.)

prop.table(table(aws$AWS_ID))
prop.table(table(aws$AWS_ID)) * 100

paste0(prop.table(table(aws$AWS_ID)) * 100, "%")

In [ ]:
### 기술통계 함수의 모듈화
# 각 컬럼 단위로 빈도와 최대 최소값 계산

test <- read.csv("../data/test.csv", header = T)
head(test)
length(test)
str(test)

table(test$A)
max(test$A)
min(test$A)

data_proc <- function(df){
  for(idx in 1:length(df)){
    cat(idx, "번째 컬럼의 빈도 분석 결과")
    print(table(df[idx]))
    cat("\n")
  }
  
  for(idx in 1:length(df)){
    f <- table(df[idx])
    cat(idx, "번째 컬럼의 최대/최소값 결과 : \t")
    cat("max =", max(f), ", min = ", min(f), "\n")
  }
}

data_proc(test)

In [ ]:
#### plyr, dplyr ####

### plyr
install.packages("plyr")
library(plyr)

x <- data.frame(id = c(1, 2, 3, 4, 5, 6), height = c(160, 171, 173, 162, 165, 170))
y <- data.frame(id = c(5, 4, 1, 3, 2, 7), weight = c(55, 73, 60, 57, 80, 91))
x
y
?plyr

In [ ]:
# 데이터 병합
xy <- join(x, y, by = "id", type = "left")
xy

xy <- join(x, y, by = "id", type = "right")
xy

xy <- join(x, y, by = "id", type = "full")
xy

xy <- join(x, y, by = "id", type = "inner")
xy

In [ ]:
# 다중 키
x <- data.frame(key1 = c(1, 1, 2, 2, 3), key2 = c('a', 'b', 'c', 'd', 'e'),
                val = c(10, 20, 30, 40, 50))

y <- data.frame(key1 = c(3, 2, 2, 1, 1), key2 = c('e', 'd', 'c', 'b', 'a'),
                val = c(500, 400, 300, 200, 100))

xy <- join(x, y, by = c("key1", "key2"))
xy

In [ ]:
# 기술 통계량
# tapply : 집단 변수를 대상으로 한번에 하나의 통계량을 구할 때 사용(기본 함수)
# ddply : 한번에 여러 개의 통계량을 구할 때 사용 (plyr 패키지 함수)

head(iris)
str(iris)
unique(iris$Species)
table(iris$Species)
class(iris$Species)

?tapply
tapply(iris$Sepal.Length, iris$Species, mean)
tapply(iris$Sepal.Length, iris$Species, sd)

?ddply
ddply(iris, .(Species), summarise, avg = mean(Sepal.Length))
ddply(iris, .(Species), summarise, avg = mean(Sepal.Length), std = sd(Sepal.Length),
      max = max(Sepal.Length), min = min(Sepal.Length))

In [ ]:
### dplyr
install.packages("dplyr")
library(dplyr)
?dplyr
help(package = dplyr)

exam <- read.csv("../data/csv_exam.csv")
exam

In [ ]:
### filter()

# 1반 학생들의 데이터 추출
exam[exam$class == 1, ]
subset(exam, class==1)

filter(exam, class==1)
exam %>% filter(class==1)

# 2반이면서 영어점수가 80점 이상인 데이터만 추출
exam[exam$class==2 & exam$english>=80, ]
exam %>% filter(class==2 & english>=80)

# 1, 3, 5반에 해당하는 데이터만 추출
exam %>% filter(class==1 | class==3 | class==5)
exam %>% filter(class %in% c(1, 3, 5))

In [ ]:
### select()
# 수학점수만 추출
exam[, 3]
exam %>% select(math)

# 반, 수학, 영어점수 추출
exam[, c(2, 3, 4)]
exam %>% select(class, math, english)

# 수학점수를 제외한 나머지 컬럼 추출
exam %>% select(-math)

# 1반 학생들의 수학점수만 추출(2명만 표시) : select math from exam there class=1 limit 2;
exam %>% filter(class==1) %>% select(class, math) %>% head(2)

exam1 <- filter(exam, class==1)
exam2 <- select(exam1, class, math)
head(exam2, n=2) # 훨씬 길고 불편하다

In [ ]:
### arrange()
exam %>% arrange(math)
exam %>% arrange(desc(math))
exam %>% arrange(class, math)

In [ ]:
### mutate()
exam$sum <- exam$math + exam$english + exam$science
exam

exam$mean <- exam$sum/3
exam

exam <- exam[ , -c(6, 7)] # 열 삭제
exam

exam %>% mutate(sum=math+english+science, mean=sum/3)
exam # 원본은 그대로다

exam <- exam %>% mutate(sum=math+english+science, mean=sum/3)
exam # 원본에 저장되었다

In [ ]:
### summarise()
exam %>% summarise(mean_math = mean(math))

In [ ]:
### groupby()
exam %>% group_by(class) %>% summarise(mean_math=mean(math), sum_math=sum(math),
                                       median_math=median(math), count=n())

In [ ]:
### left_join(), bind_rows()
test1 <- data.frame(id = c(1, 2, 3, 4 ,5), midterm = c(60, 70, 80, 90, 85))
test2 <- data.frame(id = c(1, 2, 3, 4 ,5), midterm = c(70, 83, 65, 95, 80))

left_join(test1, test2, by = "id")
bind_rows(test1, test2)

In [ ]:
#### 연습문제 ####
install.packages("ggplot2")
library(gplot2)

head(ggplot2::mpg)
str(ggplot2::mpg)
class(ggplot2::mpg)

mpg <- as.data.frame(ggplot2::mpg)
tail(mpg)
names(mpg)
dim(mpg)
View(mpg)

In [ ]:
#### Data Preprocessing ####

### 1. 데이터 탐색

# 변수명 바꾸기
df_raw <- data.frame(var1 = c(1, 2, 3), var2 = c(2, 3, 2))
df_raw

# 기본(내장)함수
df_raw1 <- df_raw # 원본은 보존하고 복사본을 이용하자
names(df_raw1) <- c("v1", "v2")
df_raw1

library(dplyr)
df_raw2 <- df_raw
df_raw2 <- rename(df_raw2, v1 = var1, v2 = var2)
df_raw2

In [ ]:
#### 2. 결측치 처리
dataset1 <- read.csv("../data/dataset.csv", header = T)
str(dataset1)
head(dataset1)
View(dataset1)

# resident : 1 ~ 5까지의 값을 갖는 명목변수로  거주지를 나타낸다
# gender : 1 ~ 2까지의 값을 갖는 명목변수로 남/녀를 나타냄
# job : 1 ~ 3까지의 값을 갖는 명목변수. 직업을 나타냄
# age : 양적변수(비율) : 2 ~ 69
# position : 1 ~ 5까지의 값을 갖는 명목변수. 직위를 나타냄
# price : 양적변수(비율) : 2.1 ~ 7.9
# survey(만족도 조사) : 1 ~ 5까지 명목변수

y <- dataset1$price
plot(y)

attach(dataset1) # 더이상 dataset1$을 붙이지 않아도 된다
plot(price)

detach(dataset1) # attach의 반대
plot(price) # dataset1$을 붙이지 않았기 때문에 값을 찾지 못한다
plot(dataset1$price)

# 결측치 확인
summary(dataset1$price)

# 결측치 제거
?sum
sum(dataset1$price, na.rm = T) # na.rm -> 결측치 제거
mean(dataset1$price, na.rm = T)

price2 <- na.omit(dataset1$price) # na.omit -> 결측값이 들어있는 행 전체를 데이터 셋에서 제거
summary(price2)

# 결측치 대체 : 0으로 대체
price3 <- ifelse(is.na(dataset1$price), 0, dataset1$price)
summary(price3)
sum(price3)
mean(price3)

# 결측치 대체 : 평균으로 대체
price4 <- ifelse(is.na(dataset1$price), mean(dataset1$price, na.rm = T), dataset1$price)
summary(price4)
sum(price4)
mean(price4)

In [ ]:
#### 3. 이상치 처리
# 양적변수(수학 연산을 할 수 있는 수치 값)와 질적변수(범주로 나누어지는 값) 구별
# 색깔(질적), 무게(양적), 소프트웨어버전(질적), 자동차년식(질적)
# 온도(양적), 스카우트 순위(질적)

# 질적변수 : 도수분포표, 분할표 => 막대 그래프(도수), 원도표, ...
table(dataset1$gender) # 이상치는 table로 쉽게 확인할 수 있다
pie(table(dataset1$gender))

# 양적변수 : 산술평균, 조화평균, 중앙값 => 히스토그램, 상자도표, 시계열도표, 산포도
summary(dataset1$price)
length(dataset1$price)
str(dataset1)

plot(dataset1$price)
boxplot(dataset1$price)

# 이상치 처리
dataset2 <- subset(dataset1, price >= 2 & price <= 8) # 범위 안쪽의 데이터들만 선별한다
length(dataset2$price)

plot(dataset2$price)
boxplot(dataset2$price)
summary(dataset2$price) # Median값과 Mean값이 비슷해졌다 -> 믿을만한 데이터가 되었다

summary(dataset2$age)
plot(dataset2$age)
boxplot(dataset2$age)

In [ ]:
#### 4. Feature Engineering
View(dataset2)

# 가독성을 위해 데이터를 변경(1. 서울, 2. 인천, 3. 대전, 4. 대구, 5. 시구군)
dataset2$resident2[dataset2$resident == 1] <- "1. 서울특별시"
dataset2$resident2[dataset2$resident == 2] <- "2. 인천광역시"
dataset2$resident2[dataset2$resident == 3] <- "3. 대전광역시"
dataset2$resident2[dataset2$resident == 4] <- "4. 대구광역시"
dataset2$resident2[dataset2$resident == 5] <- "5. 시구군"

## Binning : 척도 변경(양적 -> 질적)
# 나이 변수를 청년층(30세 이하), 중년층(31 ~ 55이하), 장년층(56 ~)
dataset2$age2[dataset2$age<=30] <- "청년층"
dataset2$age2[dataset2$age>30 & dataset2$age<=55] <- "중년층"
dataset2$age2[dataset2$age>55] <- "장년층"

View(dataset2)

In [ ]:
## 역코딩
table(dataset2$survey)
t_survey <- dataset2$survey
t_survey

c_survey <- 6-t_survey
dataset2$survey <- c_survey
View(dataset2)

In [ ]:
## Dummy : 척도 변경(질적 -> 양적)
# 거주 유형 : 단독주택(1), 다가구주택(2), 아파트(3), 오피스텔(4)
# 직업 유형 : 자영업(1), 사무직(2), 서비스(3), 전문직(4), 기타

user_data <- read.csv("../data/user_data.csv", header = T)
View(user_data)

table(user_data$house_type)

# house_type2 컬럼을 새로 추가해서 단독과 다가구는 0으로, 아파트와 오피스텔은 1로 변환
user_data$house_type2[user_data$house_type == 1 | user_data$house_type == 2] <- 0
user_data$house_type2[user_data$house_type == 3 | user_data$house_type == 4] <- 1


house_type2 <- ifelse(user_data$house_type == 1 | user_data$house_type == 2, 0, 1)
user_data$house_type2 <- house_type2

table(user_data$house_type2)

In [ ]:
## 데이터 구조 변경(wide type, long type) : melt() -> long형으로 변경, cast() -> wide형으로 변경
# reshape, reshape2, tidyr, ...
install.packages("reshape2")
library(reshape2)

str(airquality)
head(airquality) # wide type이다

ml <- melt(airquality, id.vars = c("Month", "Day")) # wide to long
View(ml)

ml2 <- melt(airquality, id.vars = c("Month", "Day"), variable.name = "climate_var", value.name = "climate_val")
View(ml2)

?cast
dc1 <- dcast(ml2, Month+Day ~ climate_var) # long to wide
View(dc1)

In [ ]:
# 예제1
data <- read.csv("../data/data.csv")
View(data)
str(data)

# 날짜별로 컬럼을 생성해서 wide하게 변경
buy_data <- dcast(data, Customer_ID ~ Date)
buy_data

buy_data1 <- dcast(data, Customer_ID ~ Date, mean)
buy_data1

# 다시 long 형으로
data_long <- melt(buy_data1, id.vars = ("Customer_ID"), variable.name = "Date", value.name = "Buy")
data_long

In [ ]:
## 예제2
data <- read.csv("../data/pay_data.csv")
View(data)

# product_type을 wide하게 변경
data1 <- dcast(data, user_id ~ product_type)
data1

data2 <- dcast(data, user_id+pay_method+price ~ product_type)
data2